In [1]:
import sys
sys.path.append("/home/ubuntu/intelliscope/modules") # go to parent dir
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda,BatchNormalization
from tensorflow.keras import backend as K
from tensorflow_addons.layers import InstanceNormalization
import tensorflow_addons as tfa
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


from intelliscope import instead_data_loaders
from sklearn.metrics import classification_report
import numpy as np
from neptunecontrib.monitoring.metrics import *


In [2]:
from tensorflow.keras.models import load_model


basepath = '/home/ubuntu/intelliscope/models/'


project       = 'HYBIM'
model_number  = '24'
path = basepath + 'Experiment(' + project + '-' + model_number + ').h5'

In [3]:
print(path)

/home/ubuntu/intelliscope/models/Experiment(HYBIM-24).h5


In [4]:
PARAMS = {
    'notebook_name':'hybrid01_imbalanced_pcg_path', 
    'name':'data_a + BALANCED => [P22]', #(P7E1) 
    'description':'PCG model retrained on Data-a', #[PCG7,ECG1]
    'tags':[ 'CASS','p2.xlarge','transfer'],#,'RAdam'], 

    'data_balanced':True,    
    
    'pretrained': True, #
    'modelload_pcg_dir':'/home/ubuntu/intelliscope/models/Experiment(PCG-22).h5', # 22 is a good one
    'modelload_ecg_dir':'/home/ubuntu/intelliscope/models/Experiment(ECG-4).h5', #
    'ecg_trainable':True,
    'pcg_trainable':True,
    
    'is_features_normalized':False, 
    'Dilation_Rate':1,
    
    ##########################
    
    'augmentation':0,
    #'opt':tfa.optimizers.RectifiedAdam(lr=1e-3),
    'opt':tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    'dropout':0.2,    
    'num_epochs': 10,
    
    
    'wavelet':'comb1',    
    'batch_size':20,  
    'validation_split':0.1,
    'test_split': 0.2,
    'dataset':'physionet2016a',
    'pcg/ecg': 'pcg+ecg',
    'pcg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/pcg/',  
    'ecg_dir':'/home/ubuntu/intelliscope/for_dataloaders/waveletcomb1/dataA/ecg/', 
    'modelsave_dir':'/home/ubuntu/intelliscope/models',
    }

if PARAMS['data_balanced']==True:
    PARAMS['pcg_dir']+='balanced'
    PARAMS['ecg_dir']+='balanced'
else:
    PARAMS['pcg_dir']+='imbalanced'        
    PARAMS['ecg_dir']+='imbalanced'        
    

In [5]:
model = load_model(path)

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 272, 462, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 272, 462, 3) 0                                            
__________________________________________________________________________________________________
model (Model)                   (None, 84)           329540      input_1[0][0]                    
__________________________________________________________________________________________________
model_1 (Model)                 (None, 3840)         47360       input_2[0][0]                    
____________________________________________________________________________________________

In [6]:
X_test,Y_test = instead_data_loaders(PARAMS,'test')

#X_test_pcg = X_test[:,:,:,0:3]/255.0
#X_test_ecg = X_test[:,:,:,3:6]/255.0



Total  388
197 Images loaded across 2 Categories. Ignored blanks  0
191 Images loaded across 2 Categories. Ignored blanks  16
372 Total After removing blanks


In [7]:
y_pred = model.predict([X_test[:,:,:,0:3],X_test[:,:,:,3:6]], batch_size=X_test.shape[0], verbose=1)

y_pred_bool = np.argmax(y_pred, axis=1)
y_test=np.argmax(Y_test,axis=1)

z = classification_report(y_test, y_pred_bool)
print(z)

372/372 [==============================] - 4s 10ms/sample
              precision    recall  f1-score   support

           0       0.85      0.92      0.88       197
           1       0.90      0.82      0.86       175

    accuracy                           0.87       372
   macro avg       0.87      0.87      0.87       372
weighted avg       0.87      0.87      0.87       372



In [8]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
threshold = 0.5

print(confusion_matrix(y_test, y_pred[:, 1] > threshold))
print(f1_score(y_test, y_pred[:, 1] > threshold))
#roc_auc_score(y_test, y_pred)



[[181  16]
 [ 32 143]]
0.8562874251497006


In [9]:
import custom_metrics

In [10]:
custom_metrics._class_metrics(y_test, y_pred[:, 1] > threshold)

{'accuracy': 0.8709677419354839,
 'precision': 0.89937106918239,
 'recall': 0.8171428571428572,
 'f1_score': 0.8562874251497006,
 'f2_score': 0.8323632130384169,
 'matthews_corrcoef': 0.7425010702848471,
 'cohen_kappa': 0.7397008659144582,
 'true_positive_rate': 0.8171428571428572,
 'true_negative_rate': 0.9187817258883249,
 'positive_predictive_value': 0.89937106918239,
 'negative_predictive_value': 0.8497652582159625,
 'false_positive_rate': 0.08121827411167512,
 'false_negative_rate': 0.18285714285714286,
 'false_discovery_rate': 0.10062893081761007}